In [63]:
import pandas as pd
import numpy as np

import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers.core import Dense, Dropout
from keras.layers import Embedding
from keras.preprocessing import sequence
from livelossplot import PlotLossesKeras
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import r2_score, roc_auc_score
from sklearn.model_selection import train_test_split
%matplotlib inline

In [56]:
archivo = "dataTraining.csv"
archivo_test = "dataTesting.csv"
train_data = pd.read_csv(archivo)
test_data = pd.read_csv(archivo_test)

In [57]:
train_data["genres"]

0                                      ['Short', 'Drama']
1                           ['Comedy', 'Crime', 'Horror']
2                      ['Drama', 'Film-Noir', 'Thriller']
3                                               ['Drama']
4                         ['Action', 'Crime', 'Thriller']
                              ...                        
7890                                ['Comedy', 'Romance']
7891                   ['Action', 'Adventure', 'Fantasy']
7892    ['Adventure', 'Musical', 'Fantasy', 'Comedy', ...
7893    ['Animation', 'Adventure', 'Drama', 'Family', ...
7894      ['Animation', 'Adventure', 'Family', 'Fantasy']
Name: genres, Length: 7895, dtype: object

In [35]:
list_genres = set(train_data["genres"].apply(lambda cell:
                                      ''.join(c for c in cell if c not in "'[]").split(', ')).sum())

In [58]:
#train_data["genres"].explode()
for a in list_genres:
    train_data[a]=0
    
print(train_data)

      Unnamed: 0  year                              title  \
0           3107  2003                               Most   
1            900  2008          How to Be a Serial Killer   
2           6724  1941                     A Woman's Face   
3           4704  1954                    Executive Suite   
4           2582  1990                      Narrow Margin   
...          ...   ...                                ...   
7890        8417  2010                 Our Family Wedding   
7891        1592  1984                Conan the Destroyer   
7892        1723  1955                             Kismet   
7893        7605  1982                 The Secret of NIMH   
7894         215  2009  Tinker Bell and the Lost Treasure   

                                                   plot  \
0     most is the story of a single father who takes...   
1     a serial killer decides to teach the secrets o...   
2     in sweden ,  a female blackmailer with a disfi...   
3     in a friday afternoon in 

In [59]:
for a in list_genres:
    train_data[a].loc[train_data["genres"].str.contains(a)]=1

#train_data.apply(lambda x: str("Mystery") in train_data["genres"], axis=1)



C:\Users\carlo\AppData\Local\Temp\ipykernel_13536\1858948565.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data[a].loc[train_data["genres"].str.contains(a)]=1


In [60]:
train_data[train_data["Mystery"]==1]

,Unnamed: 0,year,title,plot,genres,rating,Film-Noir,Documentary,Fantasy,Biography,...,Crime,Horror,War,Drama,Action,Adventure,Comedy,Music,Animation,Mystery
19,9080,1945,Pursuit to Algiers,holmes and watson are recruited in a serpentin...,"['Adventure', 'Crime', 'Mystery', 'Romance']",7.2,0,0,0,0,...,1,0,0,0,0,1,0,0,0,1
30,8067,1979,Quintet,in the distant future the world is in the grip...,"['Drama', 'Mystery', 'Sci-Fi']",5.2,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
39,7692,2004,Wicker Park,"in chicago , advertising executive and former...","['Drama', 'Mystery', 'Romance', 'Thriller']",7.0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
41,6930,2014,The Scribbler,the scribbler follows suki ( katie cassidy ) ...,"['Mystery', 'Sci-Fi', 'Thriller']",5.4,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
65,10368,1946,The Blue Dahlia,when lt . commander johnny morrison returns h...,"['Crime', 'Film-Noir', 'Mystery', 'Thriller']",7.2,1,0,0,0,...,1,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7819,5720,2014,Digging Up the Marrow,a documentary exploring genre based monster ar...,"['Biography', 'Drama', 'Fantasy', 'Horror', 'M...",5.8,0,0,1,1,...,0,1,0,1,0,0,0,0,0,1
7856,9272,2012,The Motel Life,two brothers who are forced to leave reno afte...,"['Drama', 'Mystery', 'Thriller']",6.0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
7869,10426,2006,The Da Vinci Code,the story tells the investigation started by s...,"['Mystery', 'Thriller']",6.6,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
7872,3809,2014,Veronica Mars,former teenage private eye and now an aspiring...,"['Crime', 'Drama', 'Mystery', 'Thriller']",6.8,0,0,0,0,...,1,0,0,1,0,0,0,0,0,1


In [61]:
print(train_data)

      Unnamed: 0  year                              title  \
0           3107  2003                               Most   
1            900  2008          How to Be a Serial Killer   
2           6724  1941                     A Woman's Face   
3           4704  1954                    Executive Suite   
4           2582  1990                      Narrow Margin   
...          ...   ...                                ...   
7890        8417  2010                 Our Family Wedding   
7891        1592  1984                Conan the Destroyer   
7892        1723  1955                             Kismet   
7893        7605  1982                 The Secret of NIMH   
7894         215  2009  Tinker Bell and the Lost Treasure   

                                                   plot  \
0     most is the story of a single father who takes...   
1     a serial killer decides to teach the secrets o...   
2     in sweden ,  a female blackmailer with a disfi...   
3     in a friday afternoon in 

In [64]:
vect = CountVectorizer(max_features=1000)
X_dtm = vect.fit_transform(train_data['plot'])
X_dtm.shape

(7895, 1000)

In [65]:
train_data['genres'] = train_data['genres'].map(lambda x: eval(x))
le = MultiLabelBinarizer()
y_genres = le.fit_transform(train_data['genres'])

In [66]:
X_train, X_test, y_train_genres, y_test_genres = train_test_split(X_dtm, y_genres, test_size=0.33, random_state=42)

In [67]:
##First classifier
clf = OneVsRestClassifier(RandomForestClassifier(n_jobs=-1, n_estimators=100, max_depth=10, random_state=42))
clf.fit(X_train, y_train_genres)

OneVsRestClassifier(estimator=RandomForestClassifier(max_depth=10, n_jobs=-1,
                                                     random_state=42))

In [68]:
# Predicción del modelo de clasificación
y_pred_genres = clf.predict_proba(X_test)

# Impresión del desempeño del modelo
roc_auc_score(y_test_genres, y_pred_genres, average='macro')

0.7812262183677007

In [126]:
X_test_dtm = vect.transform(test_data['plot'])

cols = ['p_Action', 'p_Adventure', 'p_Animation', 'p_Biography', 'p_Comedy', 'p_Crime', 'p_Documentary', 'p_Drama', 'p_Family',
        'p_Fantasy', 'p_Film-Noir', 'p_History', 'p_Horror', 'p_Music', 'p_Musical', 'p_Mystery', 'p_News', 'p_Romance',
        'p_Sci-Fi', 'p_Short', 'p_Sport', 'p_Thriller', 'p_War', 'p_Western']

# Predicción del conjunto de test
y_pred_test_genres = clf.predict_proba(X_test_dtm)

In [127]:
# Guardar predicciones en formato exigido en la competencia de kaggle
res = pd.DataFrame(y_pred_test_genres, index=test_data["Unnamed: 0"], columns=cols)
res.index.names = ['ID']
res.to_csv('pred_genres_text_RF.csv', index_label='ID')
res.head()

,p_Action,p_Adventure,p_Animation,p_Biography,p_Comedy,p_Crime,p_Documentary,p_Drama,p_Family,p_Fantasy,...,p_Musical,p_Mystery,p_News,p_Romance,p_Sci-Fi,p_Short,p_Sport,p_Thriller,p_War,p_Western
ID,,,,,,,,,,,,,,,,,,,,,
1,0.143030,0.101960,0.024454,0.029938,0.354552,0.138830,0.030787,0.490140,0.073159,0.101339,...,0.025069,0.063208,0.000000,0.362818,0.056648,0.008970,0.017522,0.202605,0.033989,0.018117
4,0.122624,0.085786,0.024213,0.084795,0.370949,0.216657,0.080359,0.515684,0.062976,0.067019,...,0.024734,0.060935,0.000477,0.149703,0.058190,0.014248,0.020099,0.204794,0.030438,0.018506
5,0.151364,0.110284,0.013762,0.075334,0.304837,0.448736,0.021010,0.611544,0.081741,0.169121,...,0.044538,0.261372,0.000000,0.335987,0.128505,0.001016,0.048658,0.423242,0.052693,0.025351
6,0.154448,0.125772,0.020991,0.064124,0.340779,0.140892,0.009133,0.632038,0.068287,0.063631,...,0.131074,0.088418,0.000000,0.197224,0.132208,0.001432,0.039743,0.269385,0.077607,0.017862
7,0.175143,0.210069,0.035476,0.032505,0.313850,0.243150,0.021793,0.427885,0.079781,0.143879,...,0.023859,0.090359,0.000048,0.205117,0.241663,0.002634,0.018403,0.259465,0.021569,0.017585


# Part 2

In [72]:

import nltk
import re
def clean_text(text):
    # remove backslash-apostrophe 
    text = re.sub("\'", "", text) 
    # remove everything except alphabets 
    text = re.sub("[^a-zA-Z]"," ",text) 
    # remove whitespaces 
    text = ' '.join(text.split()) 
    # convert text to lowercase 
    text = text.lower() 
    
    return text

In [73]:
train_data['clean_plot'] = train_data['plot'].apply(lambda x: clean_text(x))

In [75]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\carlo\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [77]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

# function to remove stopwords
def remove_stopwords(text):
    no_stopword_text = [w for w in text.split() if not w in stop_words]
    return ' '.join(no_stopword_text)

train_data['clean_plot'] = train_data['clean_plot'].apply(lambda x: remove_stopwords(x))

In [81]:
from sklearn.preprocessing import MultiLabelBinarizer

#train_data['genres'] = train_data['genres'].map(lambda x: eval(x))
le = MultiLabelBinarizer()
y_genres = le.fit_transform(train_data['genres'])

In [83]:
xtrain, xval, ytrain, yval = train_test_split(train_data['clean_plot'], y_genres, test_size=0.2, random_state=9)

In [85]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=10000)
# create TF-IDF features
xtrain_tfidf = tfidf_vectorizer.fit_transform(xtrain)
xval_tfidf = tfidf_vectorizer.transform(xval)

In [87]:
clf2 = OneVsRestClassifier(RandomForestClassifier(n_jobs=-1, n_estimators=100, max_depth=10, random_state=42))
clf2.fit(xtrain_tfidf, ytrain)

OneVsRestClassifier(estimator=RandomForestClassifier(max_depth=10, n_jobs=-1,
                                                     random_state=42))

In [143]:
import joblib
# Exportar modelo a archivo binario .pkl
joblib.dump(clf2, 'movie_genres.pkl', compress=3)


['movie_genres.pkl']

In [144]:
joblib.dump(tfidf_vectorizer, 'tf_idf_vectorizer.pkl', compress=3)

['tf_idf_vectorizer.pkl']

In [90]:
y_pred = clf2.predict_proba(xval_tfidf)

In [105]:
# Impresión del desempeño del modelo
roc_auc_score(yval, y_pred, average='macro')

0.8034525260869129

In [100]:
X_test_dtm = tfidf_vectorizer.transform(test_data['plot'])

cols = ['p_Action', 'p_Adventure', 'p_Animation', 'p_Biography', 'p_Comedy', 'p_Crime', 'p_Documentary', 'p_Drama', 'p_Family',
        'p_Fantasy', 'p_Film-Noir', 'p_History', 'p_Horror', 'p_Music', 'p_Musical', 'p_Mystery', 'p_News', 'p_Romance',
        'p_Sci-Fi', 'p_Short', 'p_Sport', 'p_Thriller', 'p_War', 'p_Western']

# Predicción del conjunto de test
y_pred_test_genres = clf2.predict_proba(X_test_dtm)

In [137]:
# Guardar predicciones en formato exigido en la competencia de kaggle
res2 = pd.DataFrame(y_pred_test_genres, index=test_data["Unnamed: 0"], columns=cols)
res2.index.names = ['ID']
res2.to_csv('pred_genres_text_RF2.csv', index_label='ID')
res2.head()

,p_Action,p_Adventure,p_Animation,p_Biography,p_Comedy,p_Crime,p_Documentary,p_Drama,p_Family,p_Fantasy,...,p_Musical,p_Mystery,p_News,p_Romance,p_Sci-Fi,p_Short,p_Sport,p_Thriller,p_War,p_Western
ID,,,,,,,,,,,,,,,,,,,,,
1,0.143030,0.101960,0.024454,0.029938,0.354552,0.138830,0.030787,0.490140,0.073159,0.101339,...,0.025069,0.063208,0.000000,0.362818,0.056648,0.008970,0.017522,0.202605,0.033989,0.018117
4,0.122624,0.085786,0.024213,0.084795,0.370949,0.216657,0.080359,0.515684,0.062976,0.067019,...,0.024734,0.060935,0.000477,0.149703,0.058190,0.014248,0.020099,0.204794,0.030438,0.018506
5,0.151364,0.110284,0.013762,0.075334,0.304837,0.448736,0.021010,0.611544,0.081741,0.169121,...,0.044538,0.261372,0.000000,0.335987,0.128505,0.001016,0.048658,0.423242,0.052693,0.025351
6,0.154448,0.125772,0.020991,0.064124,0.340779,0.140892,0.009133,0.632038,0.068287,0.063631,...,0.131074,0.088418,0.000000,0.197224,0.132208,0.001432,0.039743,0.269385,0.077607,0.017862
7,0.175143,0.210069,0.035476,0.032505,0.313850,0.243150,0.021793,0.427885,0.079781,0.143879,...,0.023859,0.090359,0.000048,0.205117,0.241663,0.002634,0.018403,0.259465,0.021569,0.017585


In [122]:
res2

,p_Action,p_Adventure,p_Animation,p_Biography,p_Comedy,p_Crime,p_Documentary,p_Drama,p_Family,p_Fantasy,...,p_Musical,p_Mystery,p_News,p_Romance,p_Sci-Fi,p_Short,p_Sport,p_Thriller,p_War,p_Western
ID,,,,,,,,,,,,,,,,,,,,,
1,0.146403,0.110016,0.027191,0.038849,0.362690,0.144804,0.039736,0.541229,0.074550,0.086762,...,0.037149,0.082996,0.000042,0.269102,0.068860,0.036487,0.022546,0.212701,0.030017,0.020649
4,0.137836,0.109605,0.027442,0.051369,0.345171,0.180244,0.038912,0.512560,0.074550,0.072704,...,0.027681,0.074671,0.001055,0.199228,0.068860,0.006651,0.022801,0.231268,0.036620,0.020649
5,0.163105,0.122196,0.027442,0.045504,0.337930,0.344983,0.038912,0.594983,0.080904,0.095083,...,0.027681,0.146189,0.000024,0.260399,0.075398,0.006717,0.022801,0.375036,0.031262,0.020649
6,0.144644,0.125953,0.027719,0.039593,0.325785,0.165350,0.055603,0.543277,0.074370,0.080536,...,0.033397,0.093970,0.000028,0.286645,0.081109,0.006717,0.040001,0.290960,0.057798,0.020649
7,0.166127,0.145378,0.056343,0.054650,0.355207,0.174083,0.048402,0.439607,0.076703,0.114447,...,0.027681,0.153273,0.000081,0.195608,0.218631,0.006717,0.022801,0.255787,0.030017,0.020649
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11263,0.189546,0.110771,0.027442,0.038804,0.374987,0.174690,0.038871,0.497717,0.077479,0.072704,...,0.027681,0.074671,0.000102,0.219019,0.068860,0.006717,0.022801,0.247335,0.030017,0.020202
11265,0.149865,0.132504,0.035145,0.038804,0.388764,0.142649,0.059201,0.479780,0.074550,0.076315,...,0.082165,0.075632,0.000086,0.260663,0.133122,0.006717,0.028591,0.224796,0.029662,0.020649
11269,0.138971,0.111534,0.027554,0.038804,0.410753,0.170518,0.039776,0.540827,0.085877,0.077165,...,0.054375,0.077170,0.000033,0.269569,0.068860,0.006717,0.022912,0.216471,0.030285,0.020649


## part 3


In [129]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
clf3 = OneVsRestClassifier(lr)

In [130]:
clf3.fit(xtrain_tfidf, ytrain)

OneVsRestClassifier(estimator=LogisticRegression())

In [131]:
y_pred3 = clf3.predict_proba(xval_tfidf)

In [145]:
import joblib
# Exportar modelo a archivo binario .pkl
joblib.dump(clf3, 'movie_genres_real.pkl', compress=3)

['movie_genres_real.pkl']

In [133]:
# Impresión del desempeño del modelo
roc_auc_score(yval, y_pred3, average='macro')

0.8812263836617923

In [134]:
X_test_dtm = tfidf_vectorizer.transform(test_data['plot'])

cols = ['p_Action', 'p_Adventure', 'p_Animation', 'p_Biography', 'p_Comedy', 'p_Crime', 'p_Documentary', 'p_Drama', 'p_Family',
        'p_Fantasy', 'p_Film-Noir', 'p_History', 'p_Horror', 'p_Music', 'p_Musical', 'p_Mystery', 'p_News', 'p_Romance',
        'p_Sci-Fi', 'p_Short', 'p_Sport', 'p_Thriller', 'p_War', 'p_Western']

# Predicción del conjunto de test
y_pred_test_genres3 = clf3.predict_proba(X_test_dtm)

In [136]:
# Guardar predicciones en formato exigido en la competencia de kaggle
res3 = pd.DataFrame(y_pred_test_genres3, index=test_data["Unnamed: 0"], columns=cols)
res3.index.names = ['ID']
res3.to_csv('pred_genres_text_RF3.csv', index_label='ID')
res3.head()

,p_Action,p_Adventure,p_Animation,p_Biography,p_Comedy,p_Crime,p_Documentary,p_Drama,p_Family,p_Fantasy,...,p_Musical,p_Mystery,p_News,p_Romance,p_Sci-Fi,p_Short,p_Sport,p_Thriller,p_War,p_Western
ID,,,,,,,,,,,,,,,,,,,,,
1,0.133067,0.124584,0.033603,0.035040,0.336381,0.127381,0.026928,0.566943,0.079180,0.119472,...,0.033962,0.106657,0.000918,0.528938,0.058112,0.010412,0.025240,0.179519,0.027056,0.030495
4,0.130723,0.050986,0.029530,0.120008,0.316369,0.228220,0.058147,0.735390,0.046065,0.036749,...,0.034187,0.039291,0.001026,0.112300,0.028186,0.010781,0.034896,0.223570,0.045118,0.024818
5,0.129682,0.041176,0.015846,0.047590,0.204104,0.656877,0.036500,0.700048,0.028143,0.043257,...,0.020628,0.318470,0.001116,0.129302,0.067476,0.009167,0.022839,0.488773,0.027166,0.021656
6,0.106001,0.089515,0.021637,0.055035,0.203494,0.077088,0.033174,0.656007,0.057235,0.053097,...,0.038491,0.106711,0.000937,0.193923,0.087923,0.007794,0.029242,0.280182,0.076899,0.021391
7,0.074203,0.076171,0.030783,0.028179,0.233582,0.115021,0.030421,0.346483,0.053185,0.111507,...,0.024257,0.132572,0.000938,0.162058,0.326006,0.009542,0.017376,0.445691,0.020379,0.022391


## Part 4

In [139]:
# Number of trees in random forest
n_estimators = np.linspace(100, 3000, int((3000-100)/200) + 1, dtype=int)
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [1, 5, 10, 20, 50, 75, 100, 150, 200]
# Minimum number of samples required to split a node
# min_samples_split = [int(x) for x in np.linspace(start = 2, stop = 10, num = 9)]
min_samples_split = [1, 2, 5, 10, 15, 20, 30]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 3, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Criterion
criterion=['gini', 'entropy']
random_grid = {'n_estimators': n_estimators,
#                'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap,
               'criterion': criterion}

In [140]:
from sklearn.model_selection import RandomizedSearchCV
rf_base = RandomForestClassifier()
rf_random = RandomizedSearchCV(estimator = rf_base,
                               param_distributions = random_grid,
                               n_iter = 30, cv = 5,
                               verbose=2,
                               random_state=42, n_jobs = 4)

In [141]:
##First classifier
clf4 = OneVsRestClassifier(rf_random)
clf4.fit(X_train, y_train_genres)

Fitting 5 folds for each of 30 candidates, totalling 150 fits


D:\anaconda\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
15 fits failed out of a total of 150.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "D:\anaconda\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "D:\anaconda\lib\site-packages\sklearn\ensemble\_forest.py", line 450, in fit
    trees = Parallel(
  File "D:\anaconda\lib\site-packages\joblib\parallel.py", line 1043, in __call__
    if self.dispatch_one_batch(iterator):
  File "D:\anaconda\lib\site-packages\joblib\parallel.py", line 861, in dispatch_one_batch
    self._dis

Fitting 5 folds for each of 30 candidates, totalling 150 fits


D:\anaconda\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
15 fits failed out of a total of 150.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "D:\anaconda\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "D:\anaconda\lib\site-packages\sklearn\ensemble\_forest.py", line 450, in fit
    trees = Parallel(
  File "D:\anaconda\lib\site-packages\joblib\parallel.py", line 1043, in __call__
    if self.dispatch_one_batch(iterator):
  File "D:\anaconda\lib\site-packages\joblib\parallel.py", line 861, in dispatch_one_batch
    self._dis

Fitting 5 folds for each of 30 candidates, totalling 150 fits


...

Fitting 5 folds for each of 30 candidates, totalling 150 fits


D:\anaconda\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
15 fits failed out of a total of 150.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "D:\anaconda\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "D:\anaconda\lib\site-packages\sklearn\ensemble\_forest.py", line 450, in fit
    trees = Parallel(
  File "D:\anaconda\lib\site-packages\joblib\parallel.py", line 1043, in __call__
    if self.dispatch_one_batch(iterator):
  File "D:\anaconda\lib\site-packages\joblib\parallel.py", line 861, in dispatch_one_batch
    self._dis

Fitting 5 folds for each of 30 candidates, totalling 150 fits


D:\anaconda\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
y_pred4 = clf4.predict_proba(xval_tfidf)

In [ ]:
# Impresión del desempeño del modelo
roc_auc_score(yval, y_pred4, average='macro')